# Generate Queries to test RAG System

We put the entire text of the file into context and send it to Gemini-2.5-Flash and ask it to generate a list of queries that we can use to test the RAG system.

In [3]:
from pydantic import BaseModel
from typing import List
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

class TestCase(BaseModel):
    query: str
    expected_answer: str
    citations: List[str]

class OutputSchema(BaseModel):
    test_cases: List[TestCase]

# Load document context
with open("texts/Alive's Adventures in Wonderland.txt", "r") as f:
    document_context = f.read()

# Prepare system prompt
system_prompt = """
You are a world-class assistant specializing in generating comprehensive test cases for Retrieval-Augmented Generation (RAG) systems.

Given the full context of a document, your task is to create a diverse set of at least 100 queries to rigorously evaluate the RAG system. Each query should target specific facts, details, or concepts from the context, and once specific queries are exhausted, include broader or inferential questions that still relate to the content.

For each test case, provide:
- `query`: A clear, concise question that could be asked of the RAG system.
- `expected_answer`: The precise answer that should be returned, based strictly on the context.
- `citations`: A list of references (quotes, chapter titles, or locations) from the context that support the answer.

Ensure queries cover a wide range of topics, including factual recall, reasoning, chronology, character analysis, and thematic understanding. Avoid duplication and strive for variety in question types.

Return the output as a JSON object with a list of test cases in the following format:
{{
    "test_cases": [
        {{
            "query": "...",
            "expected_answer": "...",
            "citations": ["..."]
        }},
        ...
    ]
}}

Context:
{context}
"""

# Set up Gemini LLM with LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    thinking_budget=0,
    response_mime_type="application/json",
    response_schema=OutputSchema.schema()
)

prompt = PromptTemplate(
    input_variables=["context"],
    template=system_prompt
)

chain = LLMChain(llm=llm, prompt=prompt)

# Generate the test set
result = chain.run(context=document_context)

result

/var/folders/yy/t6y4brw52tvdr8k9xbwc01d40000gn/T/ipykernel_31932/4256578781.py:61: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response_schema=OutputSchema.schema()
Key '$defs' is not supported in schema, ignoring


'{\n  "test_cases": [\n    {\n      "citations": [\n        "Title: Alice\'s Adventures in Wonderland",\n        "Author: Lewis Carroll"\n      ],\n      "expected_answer": "The title of the eBook is \\"Alice\'s Adventures in Wonderland\\" and the author is Lewis Carroll.",\n      "query": "What is the title of the eBook and who is the author?"\n    },\n    {\n      "citations": [\n        "Release date: June 27, 2008 [eBook #11]",\n        "Most recently updated: June 26, 2025"\n      ],\n      "expected_answer": "The eBook was released on June 27, 2008, and was most recently updated on June 26, 2025.",\n      "query": "When was the eBook released and when was it last updated?"\n    },\n    {\n      "citations": [\n        "Language: English"\n      ],\n      "expected_answer": "The language of the eBook is English.",\n      "query": "In what language is the eBook written?"\n    },\n    {\n      "citations": [\n        "Credits: Arthur DiBianca and David Widger"\n      ],\n      "expe

In [4]:
import json

parsed = json.loads(result)
print(parsed["test_cases"])

[{'citations': ["Title: Alice's Adventures in Wonderland", 'Author: Lewis Carroll'], 'expected_answer': 'The title of the eBook is "Alice\'s Adventures in Wonderland" and the author is Lewis Carroll.', 'query': 'What is the title of the eBook and who is the author?'}, {'citations': ['Release date: June 27, 2008 [eBook #11]', 'Most recently updated: June 26, 2025'], 'expected_answer': 'The eBook was released on June 27, 2008, and was most recently updated on June 26, 2025.', 'query': 'When was the eBook released and when was it last updated?'}, {'citations': ['Language: English'], 'expected_answer': 'The language of the eBook is English.', 'query': 'In what language is the eBook written?'}, {'citations': ['Credits: Arthur DiBianca and David Widger'], 'expected_answer': 'The credits for the eBook are Arthur DiBianca and David Widger.', 'query': 'Who are credited for this Project Gutenberg eBook?'}, {'citations': ['CHAPTER I. Down the Rabbit-Hole'], 'expected_answer': 'The first chapter i

# Turn into DataFrame

In [5]:
import pandas as pd
df = pd.DataFrame(parsed["test_cases"])
df.head()

,citations,expected_answer,query
0,"[Title: Alice's Adventures in Wonderland, Auth...","The title of the eBook is ""Alice's Adventures ...",What is the title of the eBook and who is the ...
1,"[Release date: June 27, 2008 [eBook #11], Most...","The eBook was released on June 27, 2008, and w...",When was the eBook released and when was it la...
2,[Language: English],The language of the eBook is English.,In what language is the eBook written?
3,[Credits: Arthur DiBianca and David Widger],The credits for the eBook are Arthur DiBianca ...,Who are credited for this Project Gutenberg eB...
4,[CHAPTER I. Down the Rabbit-Hole],"The first chapter is titled ""Down the Rabbit-H...",What is the title of Chapter I?


In [6]:
# Reorder the columns, query first, then expected_answer, then citations
df = df[["query", "expected_answer", "citations"]]

In [7]:
# Save the DataFrame to a CSV file
df.to_csv("longAliceTestCases.csv", index=False)